In [221]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import date, datetime
import time

## Etapes:

- sélection des capteurs qui correspondent à nos critères : colonnes speed non vide + enregistrement toutes les six minutes
- Suppression des colonnes inutiles ou pas assez fournies ('VL/h', 'PL/h', 'Unnamed: 0', 'delta', 'occupancy')
- Ajout des données météos
- Add Datepart 
- Création d'une carte avec les capteurs (non-utilisées)
- Ajout des variables à prédire


## Sélection des capteurs

In [767]:
df_n230 = pd.read_csv('data/df_n230_clean.csv')

In [768]:
df_n230 = df_n230[df_n230['date'] > '2019-09-23']

In [769]:
len(df_n230['sensor'].value_counts())

65

In [770]:
df_n230['sensor'].value_counts().head(35)

5044.14 2    6720
5044.12 2    6720
5146.11 1    6720
5442.20 2    6720
5146.12 2    6720
5742.00 2    6720
5146.10 2    6720
5143.10 2    6720
5442.21 2    6720
5044.01 2    6720
5143.00 2    6720
LAPI L42     6720
5442.00 2    6720
5043.00 1    6720
5044.10 2    6720
5143.01 1    6720
5143.11 1    6720
5044.15 1    6720
5143.01 2    6720
5043.21 2    6720
5043.20 2    6720
5044.11 1    6720
5044.13 1    6720
5342.10 1    6720
5146.13 1    6720
LAPI L41     6720
5043.15 1    6720
5146.14 2    6720
5342.00 1    6720
5042.00 2    6720
5342.11 1    6720
5143.12 2    6720
5146.15 1    6720
5042.00 1    6720
5044.01 1    6720
Name: sensor, dtype: int64

In [771]:
df_n230['sensor'].value_counts().tail(30)

5143.13 1      6720
5742.00 1      6720
S_MBG33.V_0     889
S_MB333.e_0     889
S_MB333.e_1     889
S_MBG33.M_0     889
S_MB233.V_0     889
S_MBG33.M_5     889
S_MB333.D_1     889
S_MBG33.T_1     889
S_MBG33.T_0     889
S_MBG33.n_0     889
S_MBG33.V_1     889
S_MB333.D_0     889
S_MB333.e_6     889
S_MBG33.M_3     889
S_MB233.V_1     889
S_MBG33.n_1     889
S_MB333.e_4     889
S_MBG33.N_0     889
S_MB333.M_1     889
S_MBG33.M_4     889
S_MB333.e_2     889
S_MB333.M_0     889
S_MBG33.R_0     889
S_MBG33.M_1     889
S_MBG33.R_1     889
S_MBG33.M_2     889
S_MB333.e_5     889
S_MBG33.N_1     888
Name: sensor, dtype: int64

## Supprimons les capteurs qui n'ont pas des enregistrements toutes les six minutes

In [772]:
delta = df_n230['delta'][df_n230['sensor'] == 'LAPI L42'].value_counts().index[0]

In [773]:
sensor_to_remove_delta = []
for k in range(len(df_n230['sensor'].value_counts())):
    capteur = df_n230['sensor'].value_counts().index[k]
    df = df_n230[df_n230['sensor'] == capteur]
    delta_capteur = df['delta'].value_counts().index[0]
    #print(capteur)
    #print(df['delta'].value_counts().index[0])
    if delta_capteur != delta:
        sensor_to_remove_delta.append(capteur)
print(len(sensor_to_remove_delta))

28


## Supprimons les capteurs qui n'ont pas de données pour la colonne speed 

In [774]:
len(df_n230['sensor'].value_counts())
df = df_n230[df_n230['sensor'] == df_n230['sensor'].value_counts(dropna=False).index[k]]

In [775]:
sensor_to_remove_speed = []
for k in range(len(df_n230['sensor'].value_counts())): 
    df = df_n230[df_n230['sensor'] == df_n230['sensor'].value_counts(dropna=False).index[k]]
    if len(df['speed'].value_counts(dropna = False)) == 1:
        sensor_to_remove_speed.append(df_n230['sensor'].value_counts().index[k])
print(len(sensor_to_remove_speed))

41


In [776]:
for index, sensor in enumerate(sensor_to_remove_speed):
    if sensor in sensor_to_remove_delta:
        sensor_to_remove_speed.pop(index)
sensor_to_remove = sensor_to_remove_speed + sensor_to_remove_delta
print(len(sensor_to_remove))

55


In [777]:
dirty_idx = []
for sensor in sensor_to_remove:
    df = df_n230[df_n230['sensor'] == sensor]
    for i in range(len(df)):
        dirty_idx.append(df.index[i])
print(len(dirty_idx))

124696


In [778]:
df_n230 = df_n230[~df_n230.index.isin(dirty_idx)]

Vérifions:

In [779]:
capteurs_restants = df_n230['sensor'].value_counts().index.tolist()
for capteur in capteurs_restants:
    speed_values = df_n230['speed'][df_n230['sensor'] == capteur].value_counts(dropna=False)
    print(capteur,':')
    print(speed_values)
    print('  ')

5146.11 1 :
76.0    580
75.0    528
77.0    511
74.0    460
78.0    451
       ... 
34.0      1
13.0      1
9.0       1
47.0      1
15.0      1
Name: speed, Length: 77, dtype: int64
  
5146.12 2 :
84.0     879
85.0     801
83.0     761
86.0     686
82.0     618
        ... 
102.0      1
26.0       1
99.0       1
103.0      1
36.0       1
Name: speed, Length: 102, dtype: int64
  
5044.11 1 :
56.0    801
55.0    750
57.0    710
54.0    568
58.0    505
       ... 
72.0      1
8.0       1
5.0       1
68.0      1
NaN       1
Name: speed, Length: 62, dtype: int64
  
5143.01 2 :
87.0    1268
88.0    1230
86.0    1071
89.0     907
85.0     593
        ... 
69.0       1
10.0       1
22.0       1
13.0       1
37.0       1
Name: speed, Length: 83, dtype: int64
  
5044.12 2 :
43.0    1052
44.0     995
42.0     931
41.0     777
45.0     731
40.0     524
46.0     503
39.0     305
47.0     270
38.0     170
48.0     139
37.0      88
49.0      56
36.0      43
50.0      24
35.0      16
34.0      14
51.0

A priori tous les capteurs ont bien des données différentes pour la colonne speed. Vérifions le delta :

In [780]:
capteurs_restants = df_n230['sensor'].value_counts().index.tolist()
for capteur in capteurs_restants:
    speed_values = df_n230['delta'][df_n230['sensor'] == capteur].value_counts(dropna=False)
    print(capteur,':')
    print(speed_values)
    print('  ')

5146.11 1 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
5146.12 2 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
5044.11 1 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
5143.01 2 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
5044.12 2 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
LAPI L41 :
0 days 00:06:00    6720
Name: delta, dtype: int64
  
5742.00 1 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
5044.15 1 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
5143.01 1 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
5044.10 2 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dtype: int64
  
5043.00 1 :
0 days 00:06:00       6719
-28 days +00:06:00       1
Name: delta, dt

In [781]:
anomalie = speed_values.index.tolist()[1]

In [782]:
df_n230[df_n230['delta'] == anomalie]

,Unnamed: 0,sensor,date,VL/h,PL/h,TV/h,speed,occupancy,delta
53087,82919,5442.00 2,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,-28 days +00:06:00
66527,96359,5342.00 1,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,-28 days +00:06:00
79967,109799,5042.00 1,2019-09-23 00:00:00,NaN,NaN,510.0,89.0,NaN,-28 days +00:06:00
86687,116519,5042.00 2,2019-09-23 00:00:00,NaN,NaN,620.0,87.0,NaN,-28 days +00:06:00
93407,123239,5742.00 1,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,-28 days +00:06:00
100127,129959,5742.00 2,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,-28 days +00:06:00
127007,156839,5143.00 2,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,-28 days +00:06:00
147167,176999,5143.01 1,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,-28 days +00:06:00
153887,183719,5143.01 2,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,-28 days +00:06:00
174047,203879,5043.00 1,2019-09-23 00:00:00,NaN,NaN,NaN,NaN,NaN,-28 days +00:06:00


In [783]:
capteurs_restants = df_n230['sensor'].value_counts().index.tolist()
for capteur in capteurs_restants:
    speed_values = df_n230['TV/h'][df_n230['sensor'] == capteur].value_counts(dropna=False)
    print(capteur,':')
    print(speed_values)
    print('  ')

5146.11 1 :
NaN       199
240.0      83
270.0      74
260.0      71
230.0      67
         ... 
3170.0      1
3440.0      1
3180.0      1
3220.0      1
3000.0      1
Name: TV/h, Length: 323, dtype: int64
  
5146.12 2 :
300.0     58
330.0     57
310.0     56
240.0     54
320.0     54
          ..
2960.0     1
2800.0     1
3210.0     1
3030.0     1
2830.0     1
Name: TV/h, Length: 310, dtype: int64
  
5044.11 1 :
90.0      127
80.0      123
120.0     121
110.0     120
70.0      112
         ... 
1610.0      1
1570.0      1
1490.0      1
1650.0      1
NaN         1
Name: TV/h, Length: 160, dtype: int64
  
5143.01 2 :
330.0     54
360.0     51
500.0     47
380.0     43
340.0     40
          ..
5340.0     1
5140.0     1
4910.0     1
5050.0     1
5220.0     1
Name: TV/h, Length: 503, dtype: int64
  
5044.12 2 :
60.0      148
90.0      147
70.0      144
80.0      132
40.0      129
         ... 
0.0         3
1460.0      2
1510.0      2
1540.0      1
1520.0      1
Name: TV/h, Length: 154, dty

Hormis les capteurs LAPIL, les colonnes PL/h et VL/h sont toujours vides. Supprimons les ainsi que les colonnes que nous n'utiliserons pas : Occupancy, Delta, Unnamed: 0

In [784]:
df_n230.drop(['VL/h', 'PL/h', 'Unnamed: 0', 'delta', 'occupancy'], axis = 1, inplace = True)

In [785]:
df_n230

,sensor,date,TV/h,speed
53087,5442.00 2,2019-09-23 00:00:00,NaN,NaN
53088,5442.00 2,2019-09-23 00:06:00,NaN,NaN
53089,5442.00 2,2019-09-23 00:12:00,490.0,88.0
53090,5442.00 2,2019-09-23 00:18:00,470.0,89.0
53091,5442.00 2,2019-09-23 00:24:00,420.0,86.0
...,...,...,...,...
289242,LAPI L42,2019-10-20 23:30:00,520.0,88.0
289243,LAPI L42,2019-10-20 23:36:00,690.0,85.0
289244,LAPI L42,2019-10-20 23:42:00,590.0,90.0
289245,LAPI L42,2019-10-20 23:48:00,510.0,86.0


Certaines données sont manquants. Remplissons ces données par interpollation polynomial (ordre 2) :

In [786]:
def interpolation(col, method, order = None):
    df_n230[col] = df_n230[col].astype(float)
    df_n230[col] = df_n230[col].interpolate(method = method, order = order)
    return df_n230[col]

In [787]:
for col in ['TV/h', 'speed']:
    df_n230[col] = interpolation(col,'polynomial', 2)

## Ajout des données météo

In [788]:
weather_df = pd.read_csv('polynomial_weather.csv', sep = ',')

In [789]:
weather_df.index = weather_df['date']
df_n230.index = df_n230['date']
df_n230.drop(['date'], axis = 1, inplace = True)

In [790]:
df_n230 = df_n230.join(weather_df)

In [791]:
df_n230

,sensor,TV/h,speed,date,pression au niveau de la mer,Variation de pression en 3 heures,Vitesse du vent moyen 10 mn,Température,Point de rosée,Visibilité horizontale,Nébulosité des nuages de l'étage inférieur,Variation de pression en 24 heures,Température minimale du sol sur 12 heures,Etat du sol,Hauteur de la neige fraiche,Précipitation dans la dernière heure
date,,,,,,,,,,,,,,,,
2019-09-23 00:00:00,5442.00 2,NaN,NaN,2019-09-23 00:00:00,101790.0,80.000000,2.700000,287.950000,286.450000,17710.0,8.000000,1070.0,286.45,1.0,0.0,0.0
2019-09-23 00:00:00,5342.00 1,879.449197,88.348895,2019-09-23 00:00:00,101790.0,80.000000,2.700000,287.950000,286.450000,17710.0,8.000000,1070.0,286.45,1.0,0.0,0.0
2019-09-23 00:00:00,5042.00 1,510.000000,89.000000,2019-09-23 00:00:00,101790.0,80.000000,2.700000,287.950000,286.450000,17710.0,8.000000,1070.0,286.45,1.0,0.0,0.0
2019-09-23 00:00:00,5042.00 2,620.000000,87.000000,2019-09-23 00:00:00,101790.0,80.000000,2.700000,287.950000,286.450000,17710.0,8.000000,1070.0,286.45,1.0,0.0,0.0
2019-09-23 00:00:00,5742.00 1,529.384744,81.590626,2019-09-23 00:00:00,101790.0,80.000000,2.700000,287.950000,286.450000,17710.0,8.000000,1070.0,286.45,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-20 23:54:00,5146.11 1,550.000000,71.000000,2019-10-20 23:54:00,101484.0,183.666667,2.573333,283.573333,282.656667,47163.0,4.133333,1181.0,282.79,1.0,0.0,0.0
2019-10-20 23:54:00,5146.10 2,110.000000,78.000000,2019-10-20 23:54:00,101484.0,183.666667,2.573333,283.573333,282.656667,47163.0,4.133333,1181.0,282.79,1.0,0.0,0.0
2019-10-20 23:54:00,5146.12 2,530.000000,83.000000,2019-10-20 23:54:00,101484.0,183.666667,2.573333,283.573333,282.656667,47163.0,4.133333,1181.0,282.79,1.0,0.0,0.0


## Add-Datepart

In [792]:
df_n230['date'] = df_n230.index

In [793]:
df_n230['year'] = df_n230['date'].astype(str).apply(lambda x : x[:4])
df_n230['month'] = df_n230['date'].astype(str).apply(lambda x : x[5:7])
df_n230['day'] = df_n230['date'].astype(str).apply(lambda x : x[8:10])
df_n230['hour'] = df_n230['date'].astype(str).apply(lambda x : x[10:13])
df_n230['minutes'] = df_n230['date'].astype(str).apply(lambda x : x[14:16])

In [275]:
df_n230.to_csv('data/n230_clean_weather_selected', index = True)

In [276]:
df_n230 = pd.read_csv('data/n230_clean_weather_selected')
df_n230.index = df_n230['date']
df_n230.drop(['date'], axis = 1, inplace = True)

## Création d'une carte avec les capteurs

In [173]:
df_sensors = pd.read_csv('data/Bordeaux_sensordef.csv', sep=';')

In [174]:
sensors_to_keep = df_n230['sensor'].value_counts().index.tolist()

In [175]:
dirty_idx = []
all_censors = df_sensors['sensor'].value_counts().index.tolist()
for sensor in all_censors:
    if sensor not in sensors_to_keep:
        df = df_sensors[df_sensors['sensor'] == sensor]
        for i in range(len(df)):
            dirty_idx.append(df.index[i])
print(len(dirty_idx))

109


In [176]:
df_sensors = df_sensors[~df_sensors.index.isin(dirty_idx)]

In [177]:
df_sensors[['lat','lon']]

,lat,lon
53,44.820878,-0.518471
55,44.820933,-0.518100
59,44.826092,-0.509134
60,44.826134,-0.509315
61,44.835343,-0.499863
62,44.835014,-0.500026
68,44.848842,-0.503412
72,44.853130,-0.505016
73,44.853061,-0.505165
76,44.862905,-0.503809


In [178]:
dirty_idx = [131,55,60,61,72,83,81,78,80, 84,88,91,90,93]

In [179]:
df_carto = df_sensors[~df_sensors.index.isin(dirty_idx)]
df_carto = df_carto[['lat','lon']]

In [180]:
df_carto[['lat','lon']].to_csv('test_carte_gmaps.csv')

## Ajout des variables de sorties 

In [277]:
def etat_du_trafic(x):
    if x > 80: return 0
    else: return 1

In [278]:
df = pd.DataFrame(['test'] * 22, index = df_n230.columns.tolist() + ['speed6min']).T

In [279]:
capteurs = df_n230['sensor'].value_counts().index.tolist()
for capteur in capteurs:
    df_test = df_n230[df_n230['sensor'] == capteur]
    for period in [('30min',5),('60min', 10), ('3heures', 30), ('24heures', 240)]:
        var = "congestion+"+period[0]
        shift = period[1]   
        df_test[var] = df_test['speed'].shift(periods =-shift)
    df = pd.concat([df, df_test])

/opt/anaconda3/envs/fastai-cpu-v0.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [280]:
df.drop(['speed6min'], axis = 1, inplace = True)

In [281]:
df = df[~df.index.isin([0])]
df[df['sensor'] == capteurs[7]].head(35)

,sensor,TV/h,speed,date.1,pression au niveau de la mer,Variation de pression en 3 heures,Vitesse du vent moyen 10 mn,Température,Point de rosée,Visibilité horizontale,...,Précipitation dans la dernière heure,year,month,day,hour,minutes,congestion+30min,congestion+60min,congestion+3heures,congestion+24heures
2019-09-23 00:00:00,5146.13 1,147.031444,55.467221,2019-09-23 00:00:00,101790.0,80.0,2.7,287.95,286.45,17710.0,...,0.0,2019,9,23,0,0,70.0,71.0,72.0,68.0
2019-09-23 00:06:00,5146.13 1,154.884652,62.359201,2019-09-23 00:06:00,101791.0,78.0,2.65,287.903333,286.423333,17281.666667,...,0.0,2019,9,23,0,6,69.0,64.0,72.0,70.0
2019-09-23 00:12:00,5146.13 1,160.0,67.0,2019-09-23 00:12:00,101792.0,76.0,2.6,287.856667,286.396667,16853.333333,...,0.0,2019,9,23,0,12,72.0,65.0,62.0,70.0
2019-09-23 00:18:00,5146.13 1,160.0,68.0,2019-09-23 00:18:00,101793.0,74.0,2.55,287.81,286.37,16425.0,...,0.0,2019,9,23,0,18,70.0,64.0,61.0,71.0
2019-09-23 00:24:00,5146.13 1,130.0,67.0,2019-09-23 00:24:00,101794.0,72.0,2.5,287.763333,286.343333,15996.666667,...,0.0,2019,9,23,0,24,62.0,74.0,66.0,70.0
2019-09-23 00:30:00,5146.13 1,140.0,70.0,2019-09-23 00:30:00,101795.0,70.0,2.45,287.716667,286.316667,15568.333333,...,0.0,2019,9,23,0,30,71.0,63.0,65.0,65.0
2019-09-23 00:36:00,5146.13 1,110.0,69.0,2019-09-23 00:36:00,101796.0,68.0,2.4,287.67,286.29,15140.0,...,0.0,2019,9,23,0,36,64.0,64.0,66.0,72.0
2019-09-23 00:42:00,5146.13 1,100.0,72.0,2019-09-23 00:42:00,101797.0,66.0,2.35,287.623333,286.263333,14711.666667,...,0.0,2019,9,23,0,42,65.0,69.0,70.0,69.0
2019-09-23 00:48:00,5146.13 1,110.0,70.0,2019-09-23 00:48:00,101798.0,64.0,2.3,287.576667,286.236667,14283.333333,...,0.0,2019,9,23,0,48,64.0,64.0,64.0,68.0
2019-09-23 00:54:00,5146.13 1,80.0,62.0,2019-09-23 00:54:00,101799.0,62.0,2.25,287.53,286.21,13855.0,...,0.0,2019,9,23,0,54,74.0,65.0,67.0,61.0


In [282]:
df['date'] = df.index

In [283]:
trafic_datas = df[['date','sensor', 'congestion+30min', 'congestion+60min', 'congestion+3heures', 'congestion+24heures']]

In [284]:
df_sensors['id'] = df_sensors.index

In [285]:
df_to_join = df_sensors[['id', 'sensor']]
df_to_join.index = df_to_join['sensor']
df_to_join.drop(['sensor'], axis = 1, inplace = True)

/opt/anaconda3/envs/fastai-cpu-v0.7/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [286]:
trafic_datas.index = trafic_datas['sensor']

In [287]:
trafic_datas = trafic_datas.join(df_to_join)

In [288]:
trafic_datas.index = trafic_datas['date']
trafic_datas.drop(['date'], axis=1, inplace = True)

In [289]:
trafic_datas.iloc[161030:].head(25)

,sensor,congestion+30min,congestion+60min,congestion+3heures,congestion+24heures,id
date,,,,,,
2019-10-19 23:00:00,LAPI L42,87.0,86.0,84.0,87.0,132
2019-10-19 23:06:00,LAPI L42,86.0,87.0,84.0,87.0,132
2019-10-19 23:12:00,LAPI L42,87.0,85.0,84.0,86.0,132
2019-10-19 23:18:00,LAPI L42,86.0,87.0,87.0,87.0,132
2019-10-19 23:24:00,LAPI L42,87.0,84.0,84.0,87.0,132
2019-10-19 23:30:00,LAPI L42,86.0,87.0,85.0,88.0,132
2019-10-19 23:36:00,LAPI L42,87.0,86.0,84.0,85.0,132
2019-10-19 23:42:00,LAPI L42,85.0,85.0,84.0,90.0,132
2019-10-19 23:48:00,LAPI L42,87.0,86.0,86.0,86.0,132


In [290]:
trafic_datas.tail()

,sensor,congestion+30min,congestion+60min,congestion+3heures,congestion+24heures,id
date,,,,,,
2019-10-20 23:30:00,LAPI L42,NaN,NaN,NaN,NaN,132
2019-10-20 23:36:00,LAPI L42,NaN,NaN,NaN,NaN,132
2019-10-20 23:42:00,LAPI L42,NaN,NaN,NaN,NaN,132
2019-10-20 23:48:00,LAPI L42,NaN,NaN,NaN,NaN,132
2019-10-20 23:54:00,LAPI L42,NaN,NaN,NaN,NaN,132


In [291]:
trafic_datas['date'] = trafic_datas.index

In [299]:
trafic_datas_df_n230 = trafic_datas

In [269]:
trafic_datas.drop(['sensor'], axis = 1, inplace = True)

In [270]:
order_col = ['id', 'date', 'congestion+30min', 'congestion+60min', 'congestion+3heures', 'congestion+24heures']

In [272]:
trafic_datas.columns

Index(['congestion+30min', 'congestion+60min', 'congestion+3heures',
       'congestion+24heures', 'id'],
      dtype='object')

In [273]:
col_name = ['sensor_id', 'hour', 'trafic_30min', 'trafic_60min', 'trafic_3heures', 'trafic_24heures']
trafic_datas.columns = col_name

ValueError: Length mismatch: Expected axis has 5 elements, new values have 6 elements

In [243]:
def conversion_timestamp(x):
    return int(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime("%s")) + 7200

# On ajoute 7200 secondes (2 heures pour se caler sur le bon fuseau horaire)

In [245]:
trafic_datas['hour'] = trafic_datas['hour'].apply(lambda x : conversion_timestamp(x))

/opt/anaconda3/envs/fastai-cpu-v0.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [246]:
trafic_datas.to_csv('data/trafic_datas.csv', index = False)

## Ajout des données du trafic

In [307]:
df_n230.head()

,sensor,TV/h,speed,date.1,pression au niveau de la mer,Variation de pression en 3 heures,Vitesse du vent moyen 10 mn,Température,Point de rosée,Visibilité horizontale,...,Variation de pression en 24 heures,Température minimale du sol sur 12 heures,Etat du sol,Hauteur de la neige fraiche,Précipitation dans la dernière heure,year,month,day,hour,minutes
date,,,,,,,,,,,,,,,,,,,,,
2019-09-23 00:00:00,5442.00 2,NaN,NaN,2019-09-23 00:00:00,101790.0,80.0,2.7,287.95,286.45,17710.0,...,1070.0,286.45,1.0,0.0,0.0,2019,9,23,0,0
2019-09-23 00:00:00,5342.00 1,879.449197,88.348895,2019-09-23 00:00:00,101790.0,80.0,2.7,287.95,286.45,17710.0,...,1070.0,286.45,1.0,0.0,0.0,2019,9,23,0,0
2019-09-23 00:00:00,5042.00 1,510.000000,89.000000,2019-09-23 00:00:00,101790.0,80.0,2.7,287.95,286.45,17710.0,...,1070.0,286.45,1.0,0.0,0.0,2019,9,23,0,0
2019-09-23 00:00:00,5042.00 2,620.000000,87.000000,2019-09-23 00:00:00,101790.0,80.0,2.7,287.95,286.45,17710.0,...,1070.0,286.45,1.0,0.0,0.0,2019,9,23,0,0
2019-09-23 00:00:00,5742.00 1,529.384744,81.590626,2019-09-23 00:00:00,101790.0,80.0,2.7,287.95,286.45,17710.0,...,1070.0,286.45,1.0,0.0,0.0,2019,9,23,0,0


Création d'un nouvel indice afin de facilité la fusion des deux df.

In [314]:
df_n230['pour fusion'] = df_n230.index + ' ' + df_n230['sensor']
trafic_datas_df_n230['pour fusion'] = trafic_datas_df_n230.index + ' ' + trafic_datas_df_n230['sensor']

In [316]:
df_n230['date'] = df_n230.index
trafic_datas_df_n230['date'] = trafic_datas_df_n230.index

In [317]:
df_n230.index = df_n230['pour fusion']
trafic_datas_df_n230.index = trafic_datas_df_n230['pour fusion']

In [320]:
df_n230['pour fusion'].value_counts()

2019-10-17 15:12:00 LAPI L41     1
2019-10-20 20:30:00 5044.10 2    1
2019-10-13 12:48:00 5146.13 1    1
2019-10-05 00:30:00 5742.00 2    1
2019-09-23 13:42:00 5146.10 2    1
                                ..
2019-10-04 01:00:00 5146.12 2    1
2019-10-05 10:42:00 5043.00 1    1
2019-09-25 15:48:00 5146.12 2    1
2019-10-09 20:54:00 5042.00 2    1
2019-09-26 08:30:00 5742.00 1    1
Name: pour fusion, Length: 161280, dtype: int64

In [322]:
df_n230.drop(['pour fusion'], axis = 1, inplace = True)
trafic_datas_df_n230.drop(['pour fusion'], axis = 1, inplace = True)

In [334]:
trafic_datas.drop(['sensor', 'date'], axis = 1, inplace = True)

In [336]:
df_n230 = df_n230.join(trafic_datas_df_n230)

In [337]:
df_n230.index = df_n230['date']

In [339]:
df_n230.drop(['id'], axis = 1, inplace = True)

In [344]:
df_n230 = df_n230[1:]

In [346]:
df_n230.to_csv('data/df_n230_final.csv')